# Notebook 5: 模型選擇與 RJ-MCMC

本 notebook 示範如何使用 Reversible Jump MCMC (RJ-MCMC) 進行自動模型選擇。

**學習目標：**
- 理解可逆跳躍 MCMC 用於跨維度採樣
- 學習如何自動選擇誘導點數量
- 應用 RJ-MCMC 到具有挑戰性的回歸問題
- 解釋模型複雜度的後驗分布

**主要概念：**
- **RJ-MCMC**: 在模型空間和參數空間中採樣
- **自動模型選擇**: 誘導點數量 M 是推斷的
- **貝葉斯模型平均**: 使用後驗分布進行預測


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')

import os
os.environ['JAX_PLATFORM_NAME'] = 'cpu'
import jax
jax.config.update('jax_platform_name', 'cpu')
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

from plotting_style import COLORS, PALETTES, setup_plot_style, get_figure_size, format_axes
setup_plot_style()

from infodynamics_jax.core import Phi
from infodynamics_jax.gp.kernels.params import KernelParams
from infodynamics_jax.gp.kernels.rbf import rbf as rbf_kernel
from infodynamics_jax.inference.rj import RJMCMC, RJMCMCCFG
from infodynamics_jax.inference.optimisation.vfe import make_vfe_objective

print(f"JAX version: {jax.__version__}")

Metal device set to: Apple M4
JAX version: 0.8.2


W0000 00:00:1768294989.688836 15497219 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1768294989.703742 15497219 service.cc:145] XLA service 0x12b8298d0 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1768294989.703772 15497219 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1768294989.705171 15497219 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1768294989.705185 15497219 mps_client.cc:384] XLA backend will use up to 11452776448 bytes on device 0 for SimpleAllocator.


## 1. 理解 RJ-MCMC

RJ-MCMC 在參數 θ 和模型大小 M 的聯合後驗中採樣：

$$p(\theta, M, Z | y) \propto p(y | \theta, M, Z) \cdot p(\theta) \cdot p(M) \cdot p(Z | M)$$

其中：
- **θ**: 超參數（長度尺度、信號方差、噪聲方差）
- **M**: 誘導點數量
- **Z**: 誘導點位置（索引）
- **VFE**: 變分自由能（Titsias 2009）作為似然近似

**移動類型：**
1. **Birth**: 添加新的誘導點
2. **Death**: 移除誘導點
3. **HMC**: 更新超參數（固定 M）


## 2. 生成具有挑戰性的數據

In [2]:
key = jax.random.key(42)

# 生成具有間隙的數據
N0 = 1200
X_raw = jnp.sort(jax.random.uniform(key, (N0, 1), minval=0, maxval=1.5), axis=0)
mask = (X_raw < 0.6) | (X_raw > 0.8)
X = X_raw[mask].reshape(-1, 1)

# 真實函數（具有挑戰性）
def true_function(x):
    return x * jnp.sin(20 * x) + jnp.sin(4 * x) + (x > 0.5) * 0.5

f_true = true_function(X[:, 0])
key, subkey = jax.random.split(key)
noise_std = 0.1
Y = f_true + noise_std * jax.random.normal(subkey, (len(X),))

# 測試集
X_test = jnp.linspace(0, 1.5, 500)[:, None]
f_test = true_function(X_test[:, 0])

print(f"訓練數據點數: {len(X)}")
print(f"數據間隙: [0.6, 0.8]")
print(f"真實噪聲標準差: {noise_std}")

訓練數據點數: 1050
數據間隙: [0.6, 0.8]
真實噪聲標準差: 0.1


## 3. 運行 RJ-MCMC

In [3]:
# 配置 RJ-MCMC
rjmcmc_cfg = RJMCMCCFG(
    n_steps=1000,
    burn=250,
    M_min=5,
    M_max=60,
    M_init=20,
    birth_prob=0.5,
    death_mode="rank1_last",  # 或 "local_rebuild"
    hmc_step_size=1e-2,
    hmc_n_leapfrog=8,
    hmc_prob=0.3,
)

# 創建 VFE 目標
vfe_objective = make_vfe_objective(kernel_fn=rbf_kernel, residual="fitc")

# 初始化 Phi
kernel_params = KernelParams(lengthscale=jnp.array(0.1), variance=jnp.array(1.0))
M_init = rjmcmc_cfg.M_init
key, subkey = jax.random.split(key)
Z_indices = jax.random.choice(subkey, len(X), (M_init,), replace=False)
Z_init = X[Z_indices]

phi_init = Phi(
    kernel_params=kernel_params,
    Z=Z_init,
    likelihood_params={"noise_var": jnp.array(0.01)},
    jitter=1e-5,
)

# 運行 RJ-MCMC
method = RJMCMC(cfg=rjmcmc_cfg, kernel_fn=rbf_kernel)

key, subkey = jax.random.split(key)
result = method.run(
    energy=vfe_objective,
    phi_init=phi_init,
    key=subkey,
    energy_args=(X, Y),
)

print("RJ-MCMC 完成！")
print(f"  接受率 (RJ): {result.accept_rate_rj:.2%}")
print(f"  接受率 (HMC): {result.accept_rate_hmc:.2%}")
print(f"  平均 M: {float(jnp.mean(result.M_trace)):.2f}")

IndexError: boolean index did not match shape of indexed array in index 0: got (20,), expected [60]

## 4. 分析結果

In [ ]:
# 提取後驗樣本
M_trace = np.array(result.M_trace)
energy_trace = np.array(result.energy_trace)

# 視覺化 M 的後驗分布
fig, axes = plt.subplots(2, 2, figsize=get_figure_size('wide', 1.0))

# M 軌跡
ax = axes[0, 0]
ax.plot(M_trace, lw=1, color=COLORS['primary'])
ax.axhline(rjmcmc_cfg.M_init, ls=':', lw=1.2, color=COLORS['accent'], 
          label=f'初始 M={rjmcmc_cfg.M_init}')
format_axes(ax, title='模型大小 M 軌跡', xlabel='迭代', ylabel='M', legend=True)

# M 的後驗分布
ax = axes[0, 1]
ax.hist(M_trace, bins=np.arange(M_trace.min()-0.5, M_trace.max()+1.5, 1), 
       density=True, alpha=0.8, color=COLORS['primary'], edgecolor='black')
format_axes(ax, title='M 的後驗分布', xlabel='M', ylabel='密度', legend=False)

# 能量軌跡
ax = axes[1, 0]
ax.plot(energy_trace, lw=1, color=COLORS['secondary'])
format_axes(ax, title='能量軌跡', xlabel='迭代', ylabel='能量', legend=False)

# M vs 能量
ax = axes[1, 1]
scatter = ax.scatter(M_trace, energy_trace, c=energy_trace, s=20, alpha=0.6, 
                    cmap='viridis', edgecolors='none')
format_axes(ax, title='M vs 能量', xlabel='M', ylabel='能量', legend=False)
plt.colorbar(scatter, ax=ax, label='能量')

plt.tight_layout()
plt.show()

## 總結

在本 notebook 中，我們學習了：

1. **自動複雜度控制**: RJ-MCMC 自動為每個問題選擇適當的 M
2. **對不連續性魯棒**: 很好地處理階躍函數和間隙
3. **不確定性量化**: 超參數和 M 的完整貝葉斯後驗

**配置指南：**

**對於平滑函數：**
- 較低的 M_max (20-40)
- 標準先驗

**對於不連續/複雜函數：**
- 較高的 M_max (40-80)
- 更多樣本 (n_steps > 2000)
- 更長的 burn-in

**對於更快採樣：**
- 使用 `death_mode='rank1_last'`（默認）
- 減少 `hmc_n_leapfrog` 用於 theta 更新

**對於更好的準確性：**
- 使用 `death_mode='local_rebuild'`
- 增加 `hmc_n_leapfrog` 並調整 `hmc_step_size`

**擴展：**
- 多輸出 GP 回歸（獨立輸出）
- 不同核函數（Matérn、periodic 等）
- 非高斯似然（使用 Laplace 近似）
- 流式/在線推斷（增量更新）
